In [8]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import os
import soundfile as sf

In [9]:
def calculate_energy_time(y, sr, frame_length=2048, hop_length=5096):
    # Calculer l'énergie du signal audio
    energy = librosa.feature.rms(y=y, frame_length=frame_length, hop_length=hop_length)[0]
    time = librosa.frames_to_time(np.arange(len(energy)), sr=sr, hop_length=hop_length)
    return energy, time

In [10]:
def selectionne_pics(energy, sr, hop_length):
  peaks_index = librosa.util.peak_pick(energy, pre_max=5, post_max=5, pre_avg=1000, post_avg=1000, delta=0, wait=0)
  sorted_peaks_index = sorted(peaks_index, key=lambda x: energy[x], reverse=False)

  sorted_peaks_time = librosa.frames_to_time(sorted_peaks_index, sr=sr, hop_length=hop_length)

  keep_peaks = [sorted_peaks_index[-1]]
  for i in range(len(sorted_peaks_index)-1):
    keep_peak = True
    for j in range(i+1,len(sorted_peaks_index)):
      # Si moins de 5s d'écart, on garde pas le pic
      if abs(sorted_peaks_time[i] - sorted_peaks_time[j]) < 5:
        keep_peak = False
        break
    if keep_peak:
      keep_peaks.append(sorted_peaks_index[i])

  keep_peaks = sorted(keep_peaks, key=lambda x: energy[x], reverse=True)
  return keep_peaks



In [22]:
def process_audio_file(audio_file, num_echantillons=65500, max_num_peaks=5, hop_length=5096):
  y, sr = librosa.load(audio_file)
  frame_length=int(2*sr)
  energy, time = calculate_energy_time(y, sr, frame_length=frame_length, hop_length=hop_length)
  peaks_index = selectionne_pics(energy, sr, hop_length)

  start_index = 0
  # Si on a le choix dans le nombre de pics, on prends ceux du "milieu"
  if len(peaks_index) - max_num_peaks > 1:
    start_index = min((len(peaks_index) - max_num_peaks)//2, 3)

  peaks_index = peaks_index[start_index:start_index+max_num_peaks]
  peaks_time = librosa.frames_to_time(peaks_index, sr=sr, hop_length=hop_length)
  output_folder = r"C:\Users\utilisateur\Documents\Projet\RESULTAT\\" + os.path.basename(os.path.dirname(audio_file))
  output_folder
  os.makedirs(output_folder, exist_ok=True)

  for idx, peak_time in enumerate(peaks_time):
    start_sample = max(0, int(peak_time*sr) - int(num_echantillons/2))
    end_sample = min(len(y), int(peak_time*sr) + int(num_echantillons/2))
    if (start_sample==0):
      end_sample = num_echantillons
    if end_sample == len(y):
      start_sample = len(y) - num_echantillons

    segment = y[start_sample:end_sample]

    output_filename = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(audio_file))[0]}_{idx}.ogg")
    sf.write(output_filename, segment, sr)

In [24]:
hop_length=5096
num_echantillons = 65_500
max_num_peaks = 5

def process_directory(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".ogg") or file.endswith(".wav") or file.endswith(".mp3"):
                audio_file = os.path.join(root, file)
                print("Traitement du fichier audio :", audio_file)
                process_audio_file(audio_file, num_echantillons, max_num_peaks, hop_length)

In [25]:
# Appel de la fonction pour traiter le fichier zip
process_directory(r"C:\Users\utilisateur\Documents\Projet\EN_COURS")

Traitement du fichier audio : C:\Users\utilisateur\Documents\Projet\EN_COURS\strher\XC117050.ogg
Traitement du fichier audio : C:\Users\utilisateur\Documents\Projet\EN_COURS\strher\XC119208.ogg
Traitement du fichier audio : C:\Users\utilisateur\Documents\Projet\EN_COURS\strher\XC120021.ogg
Traitement du fichier audio : C:\Users\utilisateur\Documents\Projet\EN_COURS\strher\XC143353.ogg
Traitement du fichier audio : C:\Users\utilisateur\Documents\Projet\EN_COURS\strher\XC151340.ogg
Traitement du fichier audio : C:\Users\utilisateur\Documents\Projet\EN_COURS\strher\XC151341.ogg
Traitement du fichier audio : C:\Users\utilisateur\Documents\Projet\EN_COURS\strher\XC153907.ogg
Traitement du fichier audio : C:\Users\utilisateur\Documents\Projet\EN_COURS\strher\XC173408.ogg
Traitement du fichier audio : C:\Users\utilisateur\Documents\Projet\EN_COURS\strher\XC185894.ogg
Traitement du fichier audio : C:\Users\utilisateur\Documents\Projet\EN_COURS\strher\XC204078.ogg
Traitement du fichier audio : 

In [26]:
# Copie et renomme le premier cri sauvegardé (pour se retrouver avec 1 cri par fichier)

import os
import shutil

# Chemin du dossier principal contenant les sous-dossiers
dossier_principal = r"C:\Users\utilisateur\Documents\Projet\RESULTATS\cris_extraits"
nouveau_dossier = r"C:\Users\utilisateur\Documents\Projet\RESULTATS\cris_extraits_solo"

# Parcours des sous-dossiers
for sous_dossier in os.listdir(dossier_principal):
    sous_dossier_path = os.path.join(dossier_principal, sous_dossier)
    nouveau_sous_dossier_path = os.path.join(nouveau_dossier, sous_dossier)
    # Création du dossier de destination s'il n'existe pas déjà
    if not os.path.exists(nouveau_sous_dossier_path):
        os.makedirs(nouveau_sous_dossier_path)
    
    # Vérification si le chemin est un dossier
    if os.path.isdir(sous_dossier_path):
        # Parcours des fichiers dans le sous-dossier
        for fichier in os.listdir(sous_dossier_path):
            # Vérification si le nom du fichier correspond au motif "XXXX_i"
            nom_fichier, extension = os.path.splitext(fichier)
            if '_' in nom_fichier:
                prefixe, indice = nom_fichier.split('_')
                if indice == '0':
                    # Création du nouveau nom de fichier
                    nouveau_nom = prefixe + extension
                    # Chemin du fichier source
                    ancien_chemin = os.path.join(sous_dossier_path, fichier)
                    # Chemin du fichier de destination
                    nouveau_chemin = os.path.join(nouveau_sous_dossier_path, nouveau_nom)
                    # Copie du fichier avec le nouveau nom dans le nouveau dossier
                    shutil.copy2(ancien_chemin, nouveau_chemin)
                    print(f"Fichier copié : {ancien_chemin} -> {nouveau_chemin}")


Fichier copié : C:\Users\utilisateur\Documents\Projet\RESULTAT\strher\XC117050_0.ogg -> C:\Users\utilisateur\Documents\Projet\RESULTATS\cris_extraits_solo2\strher\XC117050.ogg
Fichier copié : C:\Users\utilisateur\Documents\Projet\RESULTAT\strher\XC119208_0.ogg -> C:\Users\utilisateur\Documents\Projet\RESULTATS\cris_extraits_solo2\strher\XC119208.ogg
Fichier copié : C:\Users\utilisateur\Documents\Projet\RESULTAT\strher\XC120021_0.ogg -> C:\Users\utilisateur\Documents\Projet\RESULTATS\cris_extraits_solo2\strher\XC120021.ogg
Fichier copié : C:\Users\utilisateur\Documents\Projet\RESULTAT\strher\XC143353_0.ogg -> C:\Users\utilisateur\Documents\Projet\RESULTATS\cris_extraits_solo2\strher\XC143353.ogg
Fichier copié : C:\Users\utilisateur\Documents\Projet\RESULTAT\strher\XC151340_0.ogg -> C:\Users\utilisateur\Documents\Projet\RESULTATS\cris_extraits_solo2\strher\XC151340.ogg
Fichier copié : C:\Users\utilisateur\Documents\Projet\RESULTAT\strher\XC151341_0.ogg -> C:\Users\utilisateur\Documents\P